# TFG Tests

# Imports

In [1]:
from dotenv import load_dotenv
import os
import mariadb
import pandas as pd
import fastavro
from fastavro import reader
from azure.storage.blob import BlobServiceClient

from tfg_functions import *

## sample.py

In [2]:
load_dotenv()

alberto_storage_connection_str = os.environ["ALBERTO_STORAGE_CONNECTION_STR"]
from square_analytics.container import Container
from square_analytics.events import Events

# list blobs in the capture container
capture_container = Container("capture", alberto_storage_connection_str)
print(capture_container.list_blobs())

# download events

events = Events(capture_container)
eventsdf = events.dataframe
eventsdf

['upctevents/devevents/0/2023/10/09/17/43/05.avro', 'upctevents/devevents/0/2023/10/10/16/13/05.avro', 'upctevents/devevents/0/2023/11/09/07/28/05.avro', 'upctevents/devevents/0/2023/11/21/09/13/05.avro']
Downloaded a non empty blob: upctevents/devevents/0/2023/10/09/17/43/05.avro
Downloaded a non empty blob: upctevents/devevents/0/2023/10/10/16/13/05.avro
Downloaded a non empty blob: upctevents/devevents/0/2023/11/09/07/28/05.avro
Downloaded a non empty blob: upctevents/devevents/0/2023/11/21/09/13/05.avro
Number of downloaded events: 130


,eventname,component,action,target,objecttable,objectid,crud,edulevel,contextid,contextlevel,contextinstanceid,userid,courseid,relateduserid,anonymous,other,timecreated,domain
0,\core\event\user_loggedin,core,loggedin,user,user,2,r,0,1,10,0,2,0,None,0,"{'username': 'admin', 'extrauserinfo': []}",1686832817,tfgalberto@tfg.es
1,\core\event\user_loggedin,core,loggedin,user,user,2,r,0,1,10,0,2,0,None,0,"{'username': 'admin', 'extrauserinfo': []}",1686832817,tfgalberto@tfg.es
2,\core\event\user_loggedin,core,loggedin,user,user,2,r,0,1,10,0,2,0,None,0,"{'username': 'admin', 'extrauserinfo': []}",1686832817,tfgalberto@tfg.es
3,\core\event\user_loggedin,core,loggedin,user,user,2,r,0,1,10,0,2,0,None,0,"{'username': 'admin', 'extrauserinfo': []}",1686832817,tfgalberto@tfg.es
4,\core\event\user_loggedin,core,loggedin,user,user,2,r,0,1,10,0,2,0,None,0,"{'username': 'admin', 'extrauserinfo': []}",1686832817,tfgalberto@tfg.es
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,\core\event\user_loggedin,core,loggedin,user,user,2,r,0,1,10,0,2,0,None,0,"{'username': 'alber', 'extrauserinfo': []}",1700826916,tfgalberto@tfg.es
126,\core\event\user_loggedin,core,loggedin,user,user,2,r,0,1,10,0,3,0,None,0,"{'username': 'alber', 'extrauserinfo': []}",1700827304,tfgalberto@tfg.es
127,\core\event\user_loggedin,core,loggedin,user,user,8,r,0,1,10,0,8,0,None,0,"{'username': 'alumno4', 'extrauserinfo': []}",1700828110,tfgalberto@tfg.es
128,\assignsubmission_file\event\submission_created,assignsubmission_file,created,submission,assignsubmission_file,13,c,2,75,70,29,8,2,8,0,"{'submissionid': '23', 'submissionattempt': '0...",1700828129,tfgalberto@tfg.es


## Conexion Base de Datos

In [7]:
conn = mariadb.connect(
    user="root",
    host="localhost",
    database="bitnami_moodle"
)
cur = conn.cursor()

## Lista usuarios

In [27]:
cur.execute("SELECT * FROM mdl_user")

results = cur.fetchall()

# Get the column names
cur.execute("SHOW COLUMNS FROM mdl_user")
columns = [column[0] for column in cur.fetchall()]

# Create a dataframe
df = pd.DataFrame(results, columns=columns)

# Set the display options to show all columns
pd.set_option('display.max_columns', None)

# Print the dataframe
df

,id,auth,confirmed,policyagreed,deleted,suspended,mnethostid,username,password,idnumber,firstname,lastname,email,emailstop,phone1,phone2,institution,department,address,city,country,lang,calendartype,theme,timezone,firstaccess,lastaccess,lastlogin,currentlogin,lastip,secret,picture,description,descriptionformat,mailformat,maildigest,maildisplay,autosubscribe,trackforums,timecreated,timemodified,trustbitmask,imagealt,lastnamephonetic,firstnamephonetic,middlename,alternatename,moodlenetprofile
0,1,manual,1,0,0,0,1,guest,$2y$10$CbISmTU8xM7gPyiU0Q/gz.gm9yZSB.EjPJCqPsZ...,,Guest user,,root@localhost,0,,,,,,,,en,gregorian,,99,0,0,0,0,,,0,This user is a special user that allows read-o...,1,1,0,2,1,0,0,1695997437,0,None,None,None,None,None,None
1,2,manual,1,0,0,0,1,alber,$2y$10$Ic1FLC4bDmgUngx1d317IOf/iBC4r/.Ftas3fvL...,,Admin,User,user@example.com,0,,,,,,,,en,gregorian,,99,1695997683,1707068971,1705605019,1707068815,172.18.0.1,,0,None,1,1,0,1,1,0,0,1695997437,0,None,None,None,None,None,None
2,3,manual,1,0,1,0,1,alumno1@mail.com.1696254790,$2y$10$IC5RQwIGVSyHlj3aCOALeeOQ/cYkRvjyIcP3puE...,,Alumno,1,f6b16b945f3a595d0f4d0b9270eb0cb5,0,,,,,,,,en,gregorian,,99,1696254735,1696254735,0,1696254735,172.18.0.1,,0,,1,1,0,2,1,0,1696254677,1696254790,0,,,,,,
3,4,manual,1,0,0,0,1,profesor1,$2y$10$k6nb94FTP7p3dmobD4WydeZVFcLGBYINrtS/qAX...,,Tomás,Martínez,profesor1@mail.com,0,,,,,,,,en,gregorian,,99,1696430910,1706461845,1705605085,1706461845,172.18.0.1,,0,,1,1,0,2,1,0,1696430850,1707068994,0,,,,,,
4,5,manual,1,0,0,0,1,alumno1,$2y$10$8fQQAF1Q6x0v1QQMivEreeiTLCYCum/s88dmAx3...,,Alberto,Ros,alumno1@mail.com,0,,,,,,,,en,gregorian,,99,1696432391,1705605959,1705605045,1705605959,172.18.0.1,,0,,1,1,0,2,1,0,1696432035,1707068867,0,,,,,,
5,6,manual,1,0,0,0,1,alumno2,$2y$10$ECMy3WDgiGzp2COtmKipjerRRBoMnRtQMbiG87U...,,David,López,alumno2@mail.com,0,,,,,,,,en,gregorian,,99,1697112472,1705594553,1700823880,1705594553,172.18.0.1,,0,,1,1,0,2,1,0,1697110714,1707068896,0,,,,,,
6,7,manual,1,0,0,0,1,alumno3,$2y$10$yHtEp0O39KiwS/3sC/aZF.CGtvZjYg0yAMF2vsh...,,Fran,Hernández,alumno3@mail.com,0,,,,,,,,en,gregorian,,99,1700823926,1700823926,0,1700823926,172.18.0.1,,0,,1,1,0,2,1,0,1697110798,1707068931,0,,,,,,
7,8,manual,1,0,0,0,1,alumno4,$2y$10$pBLacGcYcL5yFiCM/VhGIuuUItzPUybI8r8g5hN...,,Juan,Redondo,alumno4@mail.com,0,,,,,,,,en,gregorian,,Europe/Madrid,1697113409,1700828110,1700824002,1700828110,172.18.0.1,,0,,1,1,0,2,1,0,1697110848,1707068971,0,,,,,,


## Lista cursos

In [12]:
cur.execute("SELECT * FROM mdl_course")

results = cur.fetchall()

cur.execute("SHOW COLUMNS FROM mdl_course")
columns = [column[0] for column in cur.fetchall()]

df = pd.DataFrame(results, columns=columns)

pd.set_option('display.max_columns', None)

df

,id,category,sortorder,fullname,shortname,idnumber,summary,summaryformat,format,showgrades,newsitems,startdate,enddate,relativedatesmode,marker,maxbytes,legacyfiles,showreports,visible,visibleold,downloadcontent,groupmode,groupmodeforce,defaultgroupingid,lang,calendartype,theme,timecreated,timemodified,requested,enablecompletion,completionnotify,cacherev,originalcourseid,showactivitydates,showcompletionconditions,pdfexportfont
0,1,0,1,Moodle Docker TFG,MDTFG,,<p>Moodle powered by Bitnami</p>,0,site,1,3,0,0,0,0,0,0,0,1,1,None,0,0,0,,,,1695997436,1696432134,0,0,0,1700827481,None,0,NaN,None
1,2,1,10002,Introducción a Data Science,IDS,,<p>Curso de Introducci&oacute;n a Data Science...,1,topics,1,5,1696460400,0,0,0,0,0,0,1,1,None,0,0,0,,,,1696431929,1696431929,0,1,0,1700827481,None,1,1.0,None
2,3,1,10001,Curso de prueba TFG,CPTFG,,,1,topics,1,5,1704844800,1736380800,0,0,0,0,0,1,1,None,0,0,0,,,,1704814955,1704814955,0,1,0,1704814961,None,1,1.0,None


## Lista alumnos matriculados en cursos

In [8]:
cur.execute(f"SELECT * FROM mdl_user_enrolments")
results = cur.fetchall()

cur.execute("SHOW COLUMNS FROM mdl_user_enrolments")
columns = [column[0] for column in cur.fetchall()]

df = pd.DataFrame(results, columns=columns)

df

,id,status,enrolid,userid,timestart,timeend,modifierid,timecreated,timemodified
0,1,0,1,2,0,0,2,1696431929,1696431929
1,2,0,1,4,1696431999,0,2,1696432073,1696432073
2,3,0,1,5,1696431999,0,2,1696432084,1696432084
3,4,0,1,6,1697196199,0,2,1697196229,1697196229
4,5,0,1,7,1697196199,0,2,1697196229,1697196229
5,6,0,1,8,1697196199,0,2,1697196229,1697196229
6,7,0,4,2,0,0,2,1704814955,1704814955
7,8,0,4,5,1704814899,0,2,1704814981,1704814981
8,9,0,4,4,1704814899,0,2,1704814981,1704814981


## Lista matriculas

In [7]:
cur.execute(f"SELECT * FROM mdl_enrol")
results = cur.fetchall()

cur.execute("SHOW COLUMNS FROM mdl_enrol")
columns = [column[0] for column in cur.fetchall()]

df = pd.DataFrame(results, columns=columns)

df

,id,enrol,status,courseid,sortorder,name,enrolperiod,enrolstartdate,enrolenddate,expirynotify,...,customchar2,customchar3,customdec1,customdec2,customtext1,customtext2,customtext3,customtext4,timecreated,timemodified
0,1,manual,0,2,0,None,0,0,0,0,...,None,None,None,None,None,None,None,None,1696431929,1696431929
1,2,guest,1,2,1,None,0,0,0,0,...,None,None,None,None,None,None,None,None,1696431929,1696431929
2,3,self,1,2,2,None,0,0,0,0,...,None,None,None,None,None,None,None,None,1696431929,1696431929
3,4,manual,0,3,0,None,0,0,0,0,...,None,None,None,None,None,None,None,None,1704814955,1704814955
4,5,guest,1,3,1,None,0,0,0,0,...,None,None,None,None,None,None,None,None,1704814955,1704814955
5,6,self,1,3,2,None,0,0,0,0,...,None,None,None,None,None,None,None,None,1704814955,1704814955


## Lista tareas

In [15]:
cur.execute(f"SELECT * FROM mdl_assign")
results = cur.fetchall()

cur.execute("SHOW COLUMNS FROM mdl_assign")
columns = [column[0] for column in cur.fetchall()]

df = pd.DataFrame(results, columns=columns)

df

,id,course,name,intro,introformat,alwaysshowdescription,nosubmissions,submissiondrafts,sendnotifications,sendlatenotifications,duedate,allowsubmissionsfromdate,grade,timemodified,requiresubmissionstatement,completionsubmit,cutoffdate,gradingduedate,teamsubmission,requireallteammemberssubmit,teamsubmissiongroupingid,blindmarking,hidegrader,revealidentities,attemptreopenmethod,maxattempts,markingworkflow,markingallocation,sendstudentnotifications,preventsubmissionnotingroup,activity,activityformat,timelimit,submissionattachments
0,10,2,mod_assign,,1,1,0,0,0,0,1697497200,1696892400,100,1696939406,0,1,0,1698102000,0,0,0,0,0,0,none,-1,0,0,1,0,,1,0,0
1,12,2,modulo_tarea,,1,1,0,0,0,0,1697497200,1696892400,100,1696944634,0,1,0,1698102000,0,0,0,0,0,0,none,-1,0,0,1,0,,1,0,0
2,13,2,Tarea propuesta 1,<p>Tarea para testear la propuesta 1</p>,1,1,0,0,0,0,1697756400,1697151600,100,1697196161,0,1,0,1698361200,0,0,0,0,0,0,none,-1,0,0,1,0,,1,0,0
3,14,2,Tarea 1 TESTS BLOQUES,,1,1,0,0,0,0,1701388800,1700784000,100,1700823556,0,1,0,1701993600,0,0,0,0,0,0,none,-1,0,0,1,0,,1,0,0
4,15,2,Tarea 2 TESTS BLOQUES,,1,1,0,0,0,0,0,1700784000,100,1700823732,0,1,0,1701993600,0,0,0,0,0,0,none,-1,0,0,1,0,,1,0,0
5,16,2,Tarea 3 TESTS BLOQUES,,1,1,0,0,0,0,1703980800,1700784000,100,1700823785,0,1,0,1704672000,0,0,0,0,0,0,none,-1,0,0,1,0,,1,0,0


## Lista tareas subidas

In [16]:
cur.execute(f"SELECT * FROM mdl_assign_submission")
results = cur.fetchall()

cur.execute("SHOW COLUMNS FROM mdl_assign_submission")
columns = [column[0] for column in cur.fetchall()]

df = pd.DataFrame(results, columns=columns)

df

,id,assignment,userid,timecreated,timemodified,timestarted,status,groupid,attemptnumber,latest
0,9,10,4,1696939406,1696939406,None,new,0,0,1
1,10,10,5,1696939430,1696939464,None,submitted,0,0,1
2,11,12,4,1696944634,1696944634,None,new,0,0,1
3,12,12,5,1696944746,1696944756,None,submitted,0,0,1
4,13,13,4,1697196161,1697196161,None,new,0,0,1
5,14,13,6,1697196305,1697196316,None,submitted,0,0,1
6,15,13,8,1697196339,1697196346,None,submitted,0,0,1
7,16,14,5,1700823829,1700823855,None,submitted,0,0,1
8,17,14,6,1700823887,1700823911,None,submitted,0,0,1
9,18,15,6,1700823903,1700823903,None,new,0,0,1


## Lista roles

In [19]:
cur.execute(f"SELECT * FROM mdl_role_assignments")
results = cur.fetchall()

cur.execute("SHOW COLUMNS FROM mdl_role_assignments")
columns = [column[0] for column in cur.fetchall()]

df = pd.DataFrame(results, columns=columns)

df

,id,roleid,contextid,userid,timemodified,modifierid,component,itemid,sortorder
0,1,3,22,2,1696431929,2,,0,0
1,2,3,22,4,1696432073,2,,0,0
2,3,5,22,5,1696432084,2,,0,0
3,4,5,22,6,1697196229,2,,0,0
4,5,5,22,7,1697196229,2,,0,0
5,6,5,22,8,1697196229,2,,0,0
6,7,3,81,2,1704814955,2,,0,0
7,8,5,81,5,1704814981,2,,0,0
8,10,3,81,4,1705601851,2,,0,0


## Lista contextos

In [12]:
cur.execute(f"SELECT * FROM mdl_context")
results = cur.fetchall()

cur.execute("SHOW COLUMNS FROM mdl_context")
columns = [column[0] for column in cur.fetchall()]

df = pd.DataFrame(results, columns=columns)

df

,id,contextlevel,instanceid,path,depth,locked
0,1,10,0,/1,1,0
1,2,50,1,/1/2,2,0
2,3,40,1,/1/3,2,0
3,4,30,1,/1/4,2,0
4,5,30,2,/1/5,2,0
5,7,80,2,/1/7,2,0
6,8,80,3,/1/8,2,0
7,9,80,4,/1/9,2,0
8,10,80,5,/1/10,2,0
9,11,80,6,/1/5/11,3,0


## Propuesta 1
### Si un profesor ha activado una entrega de tarea en un curso, ¿es posible obtener los alumnos que no han hecho la entrega todavía?

Ideas: Sacar los alumnos matriculados en el curso que queramos, y de ahi sacar quienes han subido tarea fijandonos en los eventos que sean del tipo \assignsubmission_file\event\submission_created y que tengan el ID del curso mencionado y ID de los alumnos matriculados

#### Solución

In [32]:
courseid = 2
evento = '\\assignsubmission_file\\event\\submission_created'
id_tarea = '28'

cur.execute(f"SELECT u.id, u.firstname, u.lastname, u.email FROM mdl_user_enrolments ue JOIN mdl_user u ON ue.userid = u.id JOIN mdl_enrol e ON ue.enrolid = e.id WHERE e.courseid = {courseid}")
results = cur.fetchall()

df_users_course = pd.DataFrame(results, columns=['userid', 'firstname', 'lastname', 'email'])

df_tareas_subidas = eventsdf.loc[(eventsdf['eventname'] == evento) & (eventsdf['contextinstanceid'] == id_tarea)]

df_alumnos_sin_entregar = df_users_course.loc[~df_users_course['userid'].isin(df_tareas_subidas['userid'].astype('int64'))]
df_alumnos_sin_entregar

,userid,firstname,lastname,email
0,2,Admin,User,user@example.com
1,4,Profesor,1,profesor1@mail.com
2,5,Alumno,1,alumno1@mail.com
4,7,Alumno,3,alumno3@mail.com


#### Solución paso a paso
1. Alumnos matriculados en el curso que especificamos en courseid

In [ ]:
courseid = 2

cur.execute(f"SELECT u.id, u.firstname, u.lastname, u.email FROM mdl_user_enrolments ue JOIN mdl_user u ON ue.userid = u.id JOIN mdl_enrol e ON ue.enrolid = e.id WHERE e.courseid = {courseid}")

results = cur.fetchall()

df_users_course = pd.DataFrame(results, columns=['userid', 'firstname', 'lastname', 'email'])

df_users_course

2. Tareas que se han subido a una tarea concreta id_tarea

In [ ]:
evento = '\\assignsubmission_file\\event\\submission_created'
id_tarea = '28'

df_tareas_subidas = eventsdf.loc[(eventsdf['eventname'] == evento) & (eventsdf['contextinstanceid'] == id_tarea)]
df_tareas_subidas

3. Alumnos que NO han entregado la tarea id_tarea

In [ ]:
df_alumnos_sin_entregar = df_users_course.loc[~df_users_course['userid'].isin(df_tareas_subidas['userid'].astype('int64'))]
df_alumnos_sin_entregar

## Propuesta 2
### Propuesta 1 pero para un solo alumno

Ideas: Dado el ID del alumno que nos interesa y la tarea, fijarnos si hay un evento del tipo \assignsubmission_file\event\submission_created que tenga todos esos parametros

No es necesario hacer llamada a la base de datos, solo comprobar la lista de eventos

#### Solución

In [9]:
userid = '6'
evento = '\\assignsubmission_file\\event\\submission_created'
assignmentid = '28'

tarea_alumnodf = eventsdf.loc[
    (eventsdf['userid'] == userid) & 
    (eventsdf['eventname'] == evento) &
    (eventsdf['contextinstanceid'] == assignmentid)
]

if tarea_alumnodf.empty:
    print('El alumno no ha subido la tarea')
else:
    print('El alumno ha subido la tarea')

El alumno ha subido la tarea


## Propuesta 3
### Registrar para cada alumno el último login, y presentarle al profesor una lista de los alumnos que no han hecho login desde hace 2 días

Ideas: Sacar los alumnos matriculados en el curso, después sacar todos los login desde ahora a hace 2 días, y de esos login, filtrar los alumnos que no tienen ningún evento login

#### Solución

In [29]:
courseid = 2

cur.execute(f"SELECT DISTINCT u.id, u.firstname, u.lastname, u.email FROM mdl_user_enrolments ue JOIN mdl_user u ON ue.userid = u.id JOIN mdl_enrol e ON ue.enrolid = e.id JOIN mdl_role_assignments ra ON ra.userid = u.id AND ra.roleid=5 WHERE e.courseid = {courseid}")
results = cur.fetchall()

df_users_course = pd.DataFrame(results, columns=['userid', 'firstname', 'lastname', 'email'])

two_days_ago = int((pd.Timestamp.now(tz='UTC') - pd.Timedelta(days=7)).timestamp())

evento = '\\core\\event\\user_loggedin'

df_logins = eventsdf.loc[(eventsdf['eventname'] == evento) & (eventsdf['timecreated'] > two_days_ago)]

df_alumnos_sin_logear = df_users_course.loc[~df_users_course['userid'].isin(df_logins['userid'].astype('int64'))]
df_alumnos_sin_logear

,userid,firstname,lastname,email
0,5,Alberto,Ros,alumno1@mail.com
1,6,David,López,alumno2@mail.com
2,7,Fran,Hernández,alumno3@mail.com
3,8,Juan,Redondo,alumno4@mail.com


#### Solución paso a paso

1. Primero sacar los alumnos matriculados en un curso concreto, como la propuesta 1

In [30]:
courseid = 2

cur.execute(f"SELECT DISTINCT u.id, u.firstname, u.lastname, u.email FROM mdl_user_enrolments ue JOIN mdl_user u ON ue.userid = u.id JOIN mdl_enrol e ON ue.enrolid = e.id JOIN mdl_role_assignments ra ON ra.userid = u.id AND ra.roleid=5 WHERE e.courseid = {courseid}")

results = cur.fetchall()

df_users_course = pd.DataFrame(results, columns=['userid', 'firstname', 'lastname', 'email'])

df_users_course

,userid,firstname,lastname,email
0,5,Alberto,Ros,alumno1@mail.com
1,6,David,López,alumno2@mail.com
2,7,Fran,Hernández,alumno3@mail.com
3,8,Juan,Redondo,alumno4@mail.com


2. Conseguimos los logins de ahora a 2 dias. En la tabla estan en UTC

In [ ]:
evento = '\\core\\event\\user_loggedin'

two_days_ago = int((pd.Timestamp.now(tz='UTC') - pd.Timedelta(days=2)).timestamp())

df_logins = eventsdf.loc[(eventsdf['eventname'] == evento) & (eventsdf['timecreated'] > two_days_ago)]
df_logins

3. Filtramos los que no estan y así conseguimos los que no hicieron login desde hace 2 dias

In [ ]:
df_alumnos_sin_logear = df_users_course.loc[~df_users_course['userid'].isin(df_logins['userid'].astype('int64'))]
df_alumnos_sin_logear

## Propuesta 4
### Sacar fecha del último login de un alumno

Ideas: Conociendo el ID del  alumno, sacar su último evento de login de la lista de eventos

#### Solución

In [10]:
userid = '6'
evento = '\\core\\event\\user_loggedin'

last_login = eventsdf.loc[(eventsdf['userid'] == userid) & (eventsdf['eventname'] == evento)]['timecreated'].max()
fecha = pd.to_datetime(last_login, unit='s').strftime('%d-%m-%Y %H:%M:%S')
print(f"Último login de {userid}: {fecha}")

Último login de 6: 13-10-2023 11:24:53


## Subida avro a la nube

In [ ]:
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient
import os

load_dotenv()

alberto_storage_connection_str = os.environ["ALBERTO_STORAGE_CONNECTION_STR"]

blob_service_client = BlobServiceClient.from_connection_string(alberto_storage_connection_str)

# Nombre del contenedor
container_name = "aggregate"
# Ruta del archivo .avro (relativa desde este archivo python)
file_path = "aggregate.avro"
# Nombre del archivo .avro que vamos a subir al contenedor
file_name = "aggregate.avro"
# Directorio en el que se cargará el archivo .avro en el contenedor
directory_name = f"users/1/"

blob_client = blob_service_client.get_blob_client(container=container_name, blob=directory_name + file_name)

# Cargar el archivo .avro en el contenedor
with open(file_path, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

## Creacion avro de estudiante y subida a la nube

In [24]:
event = '\\core\\event\\user_loggedin'

avro_user_schema = {
    "name": "aggregate_open_repository",
    "namespace": "aggregate_open_repository_namespace",
    "type": "record",
    "fields": [
        {
            "name": "logins",
            "type": {
                "name": "logins_columns",
                "namespace": "columns",
                "type": "array",
                "default": [],
                "items": {
                    "name": "logins_items",
                    "namespace": "items",
                    "type": "record",
                    "fields": [
                        {"name": "n_logins", "type": "int"},
                        {"name": "last_login_day", "type": "string"}
                    ],
                },
            },
        },
        {"name": "firstname", "type": "string"},
    ],
}

users_loginsdf = eventsdf.loc[(eventsdf['eventname'] == event)]
users_id = users_loginsdf['userid'].unique()

# Nombre del contenedor
container_name = "aggregate"
# Ruta del archivo .avro (relativa desde este archivo python)
file_path = "aggregate.avro"
# Nombre del archivo .avro que vamos a subir al contenedor
file_name = "aggregate.avro"

blob_service_client = BlobServiceClient.from_connection_string(alberto_storage_connection_str)

for user_id in users_id:
    user_loginsdf = eventsdf.loc[(eventsdf['userid'] == user_id)]

    cur.execute(f"SELECT u.firstname FROM mdl_user u WHERE u.id = {user_id}")
    results = cur.fetchall()
    firstname = results[0][0]
    
    n_logins = user_loginsdf.shape[0]

    last_login_day = pd.to_datetime(
        user_loginsdf.iloc[-1]['timecreated'], unit='s').strftime('%d-%m')

    logins = pd.DataFrame(
        {
            "n_logins": [n_logins],
            "last_login_day": [last_login_day]
        }
    )

    users_dict = {}
    users_dict["logins"] = logins.to_dict("records")
    users_dict["firstname"] = firstname

    with open(f"aggregate.avro", "wb") as f:
        fastavro.writer(f, avro_user_schema, [users_dict])

    # Directorio en el que se cargará el archivo .avro en el contenedor
    directory_name = f"users/{user_id}/"
    
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=directory_name + file_name)

    # Cargar el archivo .avro en el contenedor
    with open(file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)

## Creacion avro de curso y subida a la nube

In [26]:
evento = '\\core\\event\\user_loggedin'

avro_course_schema = {
    "name": "aggregate_open_repository",
    "namespace": "aggregate_open_repository_namespace",
    "type": "record",
    "fields": [
        {
            "name": "students_not_logged_week",
            "type": {
                "name": "students_not_logged_week_columns",
                "namespace": "columns",
                "type": "array",
                "default": [],
                "items": {
                    "name": "students_not_logged_week_items",
                    "namespace": "items",
                    "type": "record",
                    "fields": [
                        {"name": "firstname", "type": "string"},
                        {"name": "lastname", "type": "string"},
                        {"name": "email", "type": "string"}
                    ],
                },
            },
        },
    ],
}

# Nombre del contenedor
container_name = "aggregate"
# Ruta del archivo .avro (relativa desde este archivo python)
file_path = "aggregate.avro"
# Nombre del archivo .avro que vamos a subir al contenedor
file_name = "aggregate.avro"
blob_service_client = BlobServiceClient.from_connection_string(alberto_storage_connection_str)

# Llamada a la tabla de los cursos para obtener los IDs de estos
cur.execute("SELECT id FROM mdl_course")
results = cur.fetchall()
coursesdf = pd.DataFrame(results, columns=['id'])
courses_id = coursesdf['id'].unique()

# Por cada curso, obtenemos sus alumnos y filtramos los que no han iniciado sesion
for course_id in courses_id:
    cur.execute(f"SELECT DISTINCT u.id, u.firstname, u.lastname, u.email FROM mdl_user_enrolments ue JOIN mdl_user u ON ue.userid = u.id JOIN mdl_enrol e ON ue.enrolid = e.id JOIN mdl_role_assignments ra ON ra.userid = u.id AND ra.roleid=5 WHERE e.courseid = {course_id}")
    results = cur.fetchall()
    df_users_course = pd.DataFrame(results, columns=['userid', 'firstname', 'lastname', 'email'])

    week_ago = int((pd.Timestamp.now(tz='UTC') - pd.Timedelta(days=7)).timestamp())
    df_logins = eventsdf.loc[(eventsdf['eventname'] == evento) & (eventsdf['timecreated'] > week_ago)]

    df_alumnos_sin_logear = df_users_course.loc[~df_users_course['userid'].isin(df_logins['userid'].astype('int64'))]

    cursos_dict = {}
    cursos_dict["students_not_logged_week"] = df_alumnos_sin_logear.to_dict("records")

    with open(f"aggregate.avro", "wb") as f:
        fastavro.writer(f, avro_course_schema, [cursos_dict])

    # Directorio en el que se cargará el archivo .avro en el contenedor
    directory_name = f"courses/{course_id}/"

    blob_client = blob_service_client.get_blob_client(container=container_name, blob=directory_name + file_name)

    # Cargar el archivo .avro en el contenedor
    with open(file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)

## Tests de tfg_funciones.py

In [3]:
load_dotenv()

alberto_storage_connection_str = os.environ["ALBERTO_STORAGE_CONNECTION_STR"]
from square_analytics.container import Container
from square_analytics.events import Events

# list blobs in the capture container
capture_container = Container("capture", alberto_storage_connection_str)
# download events
events = Events(capture_container)
eventsdf = events.dataframe

Downloaded a non empty blob: upctevents/devevents/0/2023/10/09/17/43/05.avro
Downloaded a non empty blob: upctevents/devevents/0/2023/10/10/16/13/05.avro
Number of downloaded events: 59


### assign_not_submitted_list

In [4]:
courseid = 2
assignmentid = '28'

df_assign_not_submitted_list = assign_not_submitted_list(courseid, assignmentid, eventsdf)
print(f"Lista alumnos sin entregar la tarea \n{df_assign_not_submitted_list}\n")

Lista alumnos sin entregar la tarea 
   userid firstname lastname               email
0       2     Admin     User    user@example.com
1       4  Profesor        1  profesor1@mail.com
2       5    Alumno        1    alumno1@mail.com
4       7    Alumno        3    alumno3@mail.com



### assign_not_submitted

In [5]:
userid = '7'
assignmentid = '28'

print(f"{assign_not_submitted(userid, assignmentid, eventsdf)}\n")

El alumno con ID 7 no ha subido la tarea



### students_not_logged

In [13]:
courseid = 2
n_days = 5
df_students_not_logged = students_not_logged(courseid, n_days, eventsdf)
print(f"Lista alumnos sin logear en {n_days} dias \n{df_students_not_logged}\n")

Lista alumnos sin logear en 5 dias 
   userid firstname lastname               email
0       2     Admin     User    user@example.com
1       4  Profesor        1  profesor1@mail.com
2       5    Alumno        1    alumno1@mail.com
3       6    Alumno        2    alumno2@mail.com
4       7    Alumno        3    alumno3@mail.com
5       8    Alumno        4    alumno4@mail.com



### last_login_date

In [7]:
userid = '5'

last_login_date(userid, eventsdf)

'Último login del alumno con ID 5: 10-10-2023 13:32:12'